In [21]:
 !pip install nltk

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [2]:
 !pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.0 MB/s eta 0:00:0000:01


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
from keras.models import Sequential
from keras.layers import Input, Embedding,TimeDistributed, LSTM, Dropout, Bidirectional, Dense 
import random
from nltk.corpus import wordnet 
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import plot_model, to_categorical
import nltk
nltk.download('punkt')  
from nltk.corpus import wordnet 
from tqdm import tqdm
import nltk
import subprocess
import logging

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet
nltk.download('punkt')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /kaggle/working/corpora/wordnet.zip
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


replace /kaggle/working/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


True

In [ ]:
 def cosine(u, v):
     return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
 from sentence_transformers import SentenceTransformer

 embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')



In [ ]:
 train_data = pd.read_json("/kaggle/input/pizza-dataset/PIZZA_train.json", lines=True)
 train_data = train_data.sample(frac=0.01, random_state=42)
 src=train_data["train.SRC"].to_numpy()
 top_decoupled=train_data["train.TOP-DECOUPLED"].to_numpy()


In [ ]:

 logging.getLogger('sentence_transformers').setLevel(logging.ERROR)
 src_emd =[]
 count=0
 new_src=np.array([])
 new_top_decoupled=np.array([])
 for src_sent,top_sent in tqdm(zip(src,top_decoupled)):
     flag=False
     sent_embedding = embedding_model.encode(src_sent)
     for emd in src_emd:
         cos=cosine(emd,sent_embedding)
         # print(cos)
         if(cos>0.9):
             flag=True
             break
     if(not flag):
         count+=1
         src_emd.append(sent_embedding)
         new_src=np.append(new_src,src_sent)
         new_top_decoupled=np.append(new_top_decoupled,top_sent)

In [25]:
def preprosses(src,top_decoupled):
    word_tag=[]
    for src,trg in  zip(src,top_decoupled):
        out=extract_bio_tags(src,trg)
        word_tag.append(out)
    return word_tag

In [ ]:
def get_pizza_order_synonyms(word):
    """
    Get synonyms related to the 'pizza order' category.
    """
    # Keywords related to pizza ordering
    pizza_keywords = [
        "pizza", "cheese", "crust", "pepperoni", "mushroom", 
        "topping", "delivery", "order","drink","meat","chicken","size","pizza style","negation"
    ]
    
    synonyms = set()
    for syn in wordnet.synsets(word):
        # Check if the synset definition matches pizza-related keywords
        if any(keyword in syn.definition().lower() for keyword in pizza_keywords):
            for lemma in syn.lemmas():
                synonym = lemma.name().replace("_", " ").replace("-", " ").lower()
                synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
                synonyms.add(synonym)
    
    # Remove the input word from synonyms if it exists
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)

In [ ]:
def synonym_replacement(sentence,top_decoupled, num_replacements=1):
    words = sentence.split()  # Tokenize sentence
    words_to_replace = [word for word in words if get_pizza_order_synonyms(word)]  # Only consider words with synonyms

    # Shuffle the list to randomly select words for replacement
    random.shuffle(words_to_replace)

    num_replaced = 0
    for word in words_to_replace:
        if num_replaced >= num_replacements:
            break
        
        synonyms = get_pizza_order_synonyms(word)
        if synonyms:
            synonym = random.choice(synonyms)  # Pick a random synonym
            words = [synonym if w == word else w for w in words]
            num_replaced += 1
            top_decoupled=re.sub(word,synonym,top_decoupled)

    return " ".join(words),top_decoupled

In [ ]:
src=new_src
top_decoupled=new_top_decoupled
src_augment=src
top_decoupled_augment=top_decoupled
for src_itr,top_itr in tqdm(zip(src_augment,top_decoupled_augment)):
    srcc,topp=synonym_replacement(src_itr,top_itr,5)
    src=np.append(src,srcc)
    top_decoupled=np.append(top_decoupled,topp)
   
    



    
    
    


In [26]:
file_path = "/kaggle/input/dataset-65/src_65.txt"

# Read data into an array of strings
with open(file_path, "r") as file:
    src = np.array([line.strip() for line in file])

file_path = "/kaggle/input/dataset-65/top_decoupled_65.txt"

# Read data into an array of strings
with open(file_path, "r") as file:
    top_decoupled = np.array([line.strip() for line in file])

In [27]:
def extract_bio_tags(src, top_decoupled):
    

    # Parse the TOP-DECOUPLED structure
    matches = re.findall(r'\(NOT \([^()]+\) \)|\((?:[^()]+)\)|\(NOT \(COMPLEX_TOPPING \((?:.+?) \) \) \)', top_decoupled)
    ind = -1
    if random.random() < 0.5:
        ind = random.randint(0, len(src.split()) - 1)

    # Map words to BIO tags
    bio_tag_map = {}
    for i, match in enumerate(matches):
        match = re.sub(r'\(|\)', '', match)
        match = match.strip()
        words = match.split()  # Split multi-word values

        if words[0] == "NOT":
            if "COMPLEX_TOPPING" in words:
                words.remove("COMPLEX_TOPPING")
                words.remove("NOT")
                indx = words.index("TOPPING")
                quantity = words[1:indx]
                topping = words[indx + 1:]
                for i, word in enumerate(quantity):
                    if i == 0:
                        bio_tag_map[word] = "B-QUANTITY"  # Begin tag
                    else:
                        bio_tag_map[word] = "I-QUANTITY"  # Inside tag
                for i, word in enumerate(topping):
                    if i == 0:
                        bio_tag_map[word] = "B-NOT-TOPPING"  # Begin tag
                    else:
                        bio_tag_map[word] = "I-NOT-TOPPING"  # Inside tag
                continue
            else:
                tag = words[0] + "-" + words[1]
                words = words[2:]
        else:
            tag = words[0]
            words = words[1:]

        for i, word in enumerate(words):
            if i == 0:
                bio_tag_map[word] = f"B-{tag}"  # Begin tag
            else:
                bio_tag_map[word] = f"I-{tag}"  # Inside tag

    src_words = src.split()
    word_tag_pairs = []

    for idx, word in enumerate(src_words):
        # Get surrounding context words
        context_before = src_words[idx - 1] if idx - 1 >= 0 else "<START>"
        context_after = src_words[idx + 1] if idx + 1 < len(src_words) else "<END>"

        # Add word, BIO tag, POS tag, and context
        bio_tag = bio_tag_map.get(word, "O")
        word_tag_pairs.append((word, bio_tag,  context_before, context_after))

    if ind != -1:
        word_tag_pairs[ind] = ("<UNK>", word_tag_pairs[ind][1], word_tag_pairs[ind][2], word_tag_pairs[ind][3])

    return word_tag_pairs

In [28]:
sentences =preprosses(src,top_decoupled)

In [39]:

maxlen = max([len(s) for s in sentences])
words = [w[0] for sentence in sentences for w in sentence]
words = list(set(words))
words.append("<pad>")
words.append("<START>")
words.append("<END>")
n_words = len(words)
print('Number of unique words:', n_words)


Number of unique words: 348


In [40]:


tags = [w[1] for sentence in sentences for w in sentence]
tags=list(set(tags))
print(tags)
n_tags = len(tags)
print('Number of unique Tags:', n_tags)

['B-VOLUME', 'I-NOT-TOPPING', 'I-QUANTITY', 'I-NOT-STYLE', 'B-CONTAINERTYPE', 'I-STYLE', 'O', 'B-STYLE', 'B-NOT-STYLE', 'I-NUMBER', 'I-SIZE', 'B-TOPPING', 'B-QUANTITY', 'I-TOPPING', 'B-NUMBER', 'I-DRINKTYPE', 'I-VOLUME', 'I-CONTAINERTYPE', 'B-DRINKTYPE', 'B-SIZE', 'B-NOT-TOPPING']
Number of unique Tags: 21


In [41]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
print(tag2idx)

{'B-VOLUME': 0, 'I-NOT-TOPPING': 1, 'I-QUANTITY': 2, 'I-NOT-STYLE': 3, 'B-CONTAINERTYPE': 4, 'I-STYLE': 5, 'O': 6, 'B-STYLE': 7, 'B-NOT-STYLE': 8, 'I-NUMBER': 9, 'I-SIZE': 10, 'B-TOPPING': 11, 'B-QUANTITY': 12, 'I-TOPPING': 13, 'B-NUMBER': 14, 'I-DRINKTYPE': 15, 'I-VOLUME': 16, 'I-CONTAINERTYPE': 17, 'B-DRINKTYPE': 18, 'B-SIZE': 19, 'B-NOT-TOPPING': 20}


In [42]:
X_words = [[word2idx[w[0]] for w in s] for s in sentences]
X_words = sequence.pad_sequences(maxlen=maxlen + 40, sequences=X_words, padding="post", value=n_words - 1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = sequence.pad_sequences(maxlen=maxlen + 40, sequences=y, padding="post", value=tag2idx['O'])
y = np.array([to_categorical(i, num_classes=n_tags) for i in y])


X_before = [[word2idx.get(w[2]) for w in s] for s in sentences]
X_before = sequence.pad_sequences(maxlen=maxlen + 40, sequences=X_before, padding="post", value=n_words - 1)

X_after = [[word2idx.get(w[3]) for w in s] for s in sentences]
X_after = sequence.pad_sequences(maxlen=maxlen + 40, sequences=X_after, padding="post", value=n_words - 1)


# Combining X_words and X_pos into a single feature input
X_combined = np.concatenate([
    np.expand_dims(X_words, axis=-1),  
    np.expand_dims(X_before, axis=-1),  
    np.expand_dims(X_after, axis=-1),  
], axis=-1)

print('X_combined shape:', X_combined.shape)



X_combined shape: (15704, 62, 3)


In [43]:
class config():
    VOCAB = n_words
    MAX_LEN = maxlen+40
    N_OUPUT = n_tags
    
    
    EMBEDDING_VECTOR_LENGTH = 256
    N_LSTM_CELLS = 150
    RECURRENT_DROPOUT = 0.4
    
    OUTPUT_ACTIVATION = 'softmax'
    
    LOSS = 'categorical_crossentropy'
    OPTIMIZER = 'adam'
    METRICS = ['accuracy']
    
    MAX_EPOCHS = 5
    

In [46]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7)
filepath = "model.keras"
ckpt = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
rlp = ReduceLROnPlateau(monitor='loss', patience=3, verbose=1)

model = Sequential()
# model.add(
#     Embedding(
#         input_dim=config.VOCAB, output_dim=config.EMBEDDING_VECTOR_LENGTH, input_length=config.MAX_LEN
#     )
# )
model.add(
    Bidirectional(
        LSTM(config.N_LSTM_CELLS, return_sequences=True, recurrent_dropout=config.RECURRENT_DROPOUT)
    )
)
model.add(
    TimeDistributed(
        Dense(config.N_OUPUT, activation=config.OUTPUT_ACTIVATION)
    )
)
model.compile(loss=config.LOSS, optimizer=config.OPTIMIZER, metrics=config.METRICS)



In [47]:
history = model.fit(x=X_combined, y=y, validation_split=0.1,
    callbacks=[es, ckpt, rlp], epochs=config.MAX_EPOCHS
)

Epoch 1/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.8816 - loss: 0.4754
Epoch 1: loss improved from inf to 0.29292, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 61s 118ms/step - accuracy: 0.8817 - loss: 0.4750 - val_accuracy: 0.9458 - val_loss: 0.1768 - learning_rate: 0.0010
Epoch 2/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.9428 - loss: 0.1820
Epoch 2: loss improved from 0.29292 to 0.17047, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 53s 120ms/step - accuracy: 0.9428 - loss: 0.1820 - val_accuracy: 0.9559 - val_loss: 0.1390 - learning_rate: 0.0010
Epoch 3/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.9535 - loss: 0.1466
Epoch 3: loss improved from 0.17047 to 0.14081, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 52s 117ms/step - accuracy: 0.9535 - loss: 0.1466 - val_accuracy: 0.9645 - val_loss: 0.1141 - learning_rate: 0.0010
Epoch 4/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.9596 - loss: 0.1

In [48]:
dev_data = pd.read_json("/kaggle/input/pizza-dataset/PIZZA_dev.json", lines=True)
src_test=dev_data["dev.SRC"].to_numpy()
top_test=dev_data["dev.TOP"].to_numpy()


In [49]:
def evaluate(test_src, test_top):
    true_len = 0
    correct = 0

    for src, top in zip(test_src, test_top):
        src_words_original = src.split()
        if len(src_words_original) > config.MAX_LEN:
            continue

        true_len += 1

        # Replace unknown words with <UNK>
        for word in src_words_original:
            if word not in word2idx:
                pattern = rf"(?<=\s){word}(?=\s)"
                src = re.sub(pattern, "<UNK>", src)

        # Prepare word-tag pairs with context for evaluation
        gold_tags = extract_bio_tags(src, top)

        # Convert words to indices
        src_words = [w[0] for w in gold_tags]
        src_indices = [word2idx.get(word, word2idx.get("<UNK>")) for word in src_words]

        # Prepare context indices
        context_before_indices = [word2idx.get(w[2], word2idx.get("<START>")) for w in gold_tags]
        context_after_indices = [word2idx.get(w[3], word2idx.get("<END>")) for w in gold_tags]

        # Pad to max length
        pad_token = word2idx.get("<pad>")
        src_indices = src_indices + [pad_token] * (config.MAX_LEN - len(src_indices))
        context_before_indices = context_before_indices + [pad_token] * (config.MAX_LEN - len(context_before_indices))
        context_after_indices = context_after_indices + [pad_token] * (config.MAX_LEN - len(context_after_indices))

        # Combine word and context indices
        src_combined = np.stack([
            src_indices,
            context_before_indices,
            context_after_indices
        ], axis=-1)
        src_combined = np.expand_dims(src_combined, axis=0)  # Add batch dimension

        # Get predictions from the model
        predictions = model.predict(src_combined)
        predictions = np.argmax(predictions, axis=-1)  # Convert to tag indices

        # Map predictions back to tags
        idx2tag = {v: k for k, v in tag2idx.items()}
        predicted_tags = [idx2tag[i] for i in predictions[0][:len(src_words_original)]]

        # Extract gold BIO tags for comparison
        gold_bio_tags = [w[1] for w in gold_tags]

        # Compare predictions to gold labels
        if predicted_tags == gold_bio_tags:
            correct += 1

    print(f"Accuracy: {correct / true_len * 100:.2f}%")

In [ ]:
evaluate(src_test,top_test)

In [ ]:

import json

with open("word2idx_model1.json", "w") as file:
    json.dump(word2idx,file)
with open("tag2idx_model1.json", "w") as file:
    json.dump(tag2idx,file)